In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 실습1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.

# 실습2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오.
# 예) 판교에 있는 괜찮은 이탈리안 식당을 몇개 추천해줄래?

import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# 실습1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.

import yfinance as yf
from io import StringIO
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import openai

end = datetime.now()
start = end - relativedelta(years=1)

df = yf.download("NVDA", start, end)
df.columns = ['Close','High','Low','Open','Volume']

csv_buffer = StringIO()
df.to_csv(csv_buffer)

csv_data = csv_buffer.getvalue()

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
  model = "gpt-4o-mini",
  messages = [
    {
      "role": "system",
      "content": "다음은 엔비디아의 날짜별 주가 데이터야. 이 데이터를 분석해서 보고해줘. 마크다운 기호는 없애고 텍스트로만 출력해줘"
    },
    {
      "role": "user",
      "content": csv_data
    }
  ],
  temperature = 0,
  max_tokens = 1024,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0
)

print(response.choices[0].message.content)

In [ ]:
# 실습2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오.
# 예) 판교에 있는 괜찮은 이탈리안 식당을 몇개 추천해줄래?

import os
import openai
import os
import json
import sys
import urllib.request

client_id = "cWCyG67A6biwOuThF11b"
client_secret = "itwHxt2ACJ"
openai.api_key = os.getenv("OPENAI_API_KEY")

question = input("📢 나에게 질문해봐")

response = openai.chat.completions.create(
  model = "gpt-3.5-turbo",
  messages = [
    {
      "role": "system",
      "content": """사용자의 질문에서 지역명(location), 업종(category), 갯수(count)를 추출해 JSON 형식으로 응답하세요.
                    지역명을 동까지 해서 더 자세히 뽑아서 응답하세요.
                    location이나 category에 여러 개가 있는 경우 ,(쉼표)로 구분해주세요.
                    예:{\"location\": \"동탄 석우동,부산 당감동\", \"category\": \"맛집,카페\", \"count\": 5}
                 """
    },
    {
      "role": "user",
      "content": question
    }
  ],
  temperature = 0,
  max_tokens = 1024,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0
)

parsed = json.loads(response.choices[0].message.content)
locations = [loc.strip() for loc in parsed.get("location", "").split(",")]
categories = [cat.strip() for cat in parsed.get("category", "").split(",")]
count = parsed.get("count", 3)

def search_naver_local(query, count):
    encText = urllib.parse.quote(query)
    url = f"https://openapi.naver.com/v1/search/local.json?query={encText}&display={count}"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))
        return data.get("items", [])
    else:
        return []

for loc in locations:
    for cat in categories:
        keyword = f"{loc} {cat}"
        print(f"\n🔍 {keyword} 검색 결과:")
        results = search_naver_local(keyword, count)
        if not results:
            print("  ❌ 결과 없음")
        for i, item in enumerate(results, 1):
            print(f"  [{i}] {item['title'].strip('<b>').strip('</b>')}")
            print(f"     주소: {item['address']}")
            print(f"     링크: {item['link']}")

In [ ]:
# 실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여, 
# 다른 종목의 데이터를 정해진 내용대로(종목, 시작날짜, 종료날짜) 가져와서 답하는 어플리케이션을 작성하시오.

def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

messages = [{"role": "user", "content": "지금 서울날씨를 섭씨로 알려줘."}]

functions = [
    {
        "name": "get_current_weather",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역이름. 예) 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
# 실습1: gradio의 chatbot interface를 사용하여 ChatGPT와 스트리밍하면서 대화하는 인터페이스를 구성하시오.
import gradio as gr
import os
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

def bot(history):
    user_message = history[-1][0]  # 마지막 사용자 입력

    # ✅ GPT에게 메시지 전달
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 또는 "gpt-4" 등
        messages=[
            {"role": "system", "content": "당신은 친절한 AI 챗봇입니다."},
            {"role": "user", "content": user_message}
        ],
        stream=True  # ⏳ 스트리밍으로 출력
    )

    history[-1][1] = ""  # 답변 초기화

    for chunk in response:
        if 'choices' in chunk and len(chunk['choices']) > 0:
            delta = chunk['choices'][0]['delta']
            if 'content' in delta:
                history[-1][1] += delta['content']
                yield history  # 실시간 업데이트


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')

In [ ]:
# 실습2: 만든 어플리케이션을 멀티턴으로 수정하시오.
import gradio as gr
import os
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

def bot(history):
    # 전체 history → GPT 메시지 포맷으로 변환
    messages = [{"role": "system", "content": "당신은 친절한 AI 챗봇입니다."}]
    for user, assistant in history[:-1]:
        messages.append({"role": "user", "content": user})
        if assistant is not None:
            messages.append({"role": "assistant", "content": assistant})
    
    # 마지막 사용자 메시지만 추가
    messages.append({"role": "user", "content": history[-1][0]})

    # GPT 호출
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        stream=True
    )

    history[-1][1] = ""
    for chunk in response:
        if "choices" in chunk and len(chunk["choices"]) > 0:
            delta = chunk["choices"][0]["delta"]
            if "content" in delta:
                history[-1][1] += delta["content"]
                yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("chat.jpg"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [6]:
# 실습: langchain을 이용하여 사용자가 업로드한 pdf 파일을 근거로 답변하는 ChatGPT 어플리케이션을 작성하시오.
# !pip install -qU pypdf

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

loader = PyPDFLoader("소나기.pdf")

documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
#embeddings = HuggingFaceEmbeddings() # sentence-transformers/all-mpnet-base-v2

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding = embeddings,
).from_loaders([loader])

index.query("소녀와 김소연은 어떤 관계인가?", llm = model, verbose = True)



> Entering new RetrievalQA chain...

> Finished chain.


'소녀와 김소연의 관계에 대한 정보는 주어진 문맥에 포함되어 있지 않습니다. 따라서 이 두 인물 간의 관계에 대해 알 수 없습니다.'